The goal is to get all the paths from a directory (and its subdirs) in dropbox, then propose for each a new location and a new filename.


# Imports

In [3]:
import os

from utils.filename_reader import create_filename_dict, remove_extension
import json

import sys
import dropbox
from dropbox.files import WriteMode
from dropbox.exceptions import ApiError, AuthError

from tokens import ACCESS_TOKEN

# Add OAuth2 access token here.
# You can generate one for yourself in the App Console.
# See <https://blogs.dropbox.com/developers/2014/05/generate-an-access-token-for-your-own-account/>
TOKEN = ACCESS_TOKEN


# API token

In [2]:
if (len(TOKEN) == 0):
    sys.exit("ERROR: Looks like you didn't add your access token. "
        "Open up backup-and-restore-example.py in a text editor and "
        "paste in your token in line 14.")

    # Create an instance of a Dropbox class, which can make requests to the API.
print("Creating a Dropbox object...")
with dropbox.Dropbox(TOKEN) as dbx:

    # Check that the access token is valid
    try:
        dbx.users_get_current_account()
    except AuthError:
        sys.exit("ERROR: Invalid access token; try re-generating an "
            "access token from the app console on the web.")
    
    

Creating a Dropbox object...


# Get the paths

In [ ]:
# DEPRECATED

def get_all_paths(dir):
    all_paths = []
    for entry in dbx.files_list_folder(dir).entries:
        if type(entry) == dropbox.files.FolderMetadata:
            all_paths += get_all_paths(entry.path_display)
        else:
            all_paths.append(entry.path_display)
    return all_paths



In [1]:
from utils.dropbox_filesystem import get_all_paths

In [5]:
l = get_all_paths('/source/REEVOID_PILOT_01_MRI_NIFTI', TOKEN)
for path in l:
    print(path)
print('\n')

l = get_all_paths('/source/REEVOID_PILOT_01_MRI_NIFTI/', TOKEN)
for path in l:
    print(path)
print('\n')

l = get_all_paths('/source/', TOKEN)
for path in l:
    print(path)

Creating a Dropbox object...
Creating a Dropbox object...
/source/REEVOID_PILOT_01_MRI_NIFTI/Resting_State/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.nii.gz
/source/REEVOID_PILOT_01_MRI_NIFTI/Resting_State/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.json


Creating a Dropbox object...
Creating a Dropbox object...
/source/REEVOID_PILOT_01_MRI_NIFTI/Resting_State/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.nii.gz
/source/REEVOID_PILOT_01_MRI_NIFTI/Resting_State/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.json


Creating a Dropbox object...
Creating a Dropbox object...
Creating a Dropbox object...
/source/REEVOID_PILOT_01_MRI_NIFTI/Resting_State/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.nii.gz
/source/REEVOID_PILOT_01_MRI_NIFTI/Resting_State/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.json
/source/Reevoid_Martin_fMRI_interoperability.zip
/source/PhysioLogFiles_fMRI_Martin.zip


# Save jsons with infos and new paths

In [ ]:
# DEPRECATED
participants_dict = {
    "REEVOID_001": "sub-01",
    "REEVOID_PILOT_01": "sub-pilot"
}

final_data = {}

for file in l:
    print(file)
    final_data[file] = create_filename_dict(file, participants_dict, sub='sub_pilot',type='func', category='resting_state')
print('done')

with open('file_infos/test.json', 'w') as f:
    json.dump(final_data, f, indent=4)

In [9]:
from utils.save_logs import save_file_infos

input_files = get_all_paths('/source/', TOKEN)

save_file_infos(
    input_files,
    participants_dict={
        "REEVOID_001": "sub-01",
        "REEVOID_PILOT_01": "sub-pilot"
    },
    out_path='file_infos/test-01.json',
    sub='sub_pilot',type='func', category='resting_state'
    )

Creating a Dropbox object...
Creating a Dropbox object...
Creating a Dropbox object...
/source/REEVOID_PILOT_01_MRI_NIFTI/Resting_State/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.nii.gz
/source/REEVOID_PILOT_01_MRI_NIFTI/Resting_State/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.json
/source/Reevoid_Martin_fMRI_interoperability.zip
/source/PhysioLogFiles_fMRI_Martin.zip
done


## Correct new paths for localizer/other metadata

In [ ]:
file_infos_path = 'file_infos/test.json'
out_path = 'file_infos/jsons_to_data/test.json'

jsons_dict = {}

data_dict = {}

out_dict = {}

with open(file_infos_path, 'r') as f:
    file_infos = json.load(f)
    all_files = file_infos.keys()
    for file in all_files:
        if file_infos[file]['extension'] == '.json':
            jsons_dict[file] = file_infos[file]
        else:
            data_dict[file] = file_infos[file]

for json_file in jsons_dict.keys():
    json_filename = json_file.split('/')[-1][:-len('.json')]
    out_dict[json_file] = []
    for data_file in data_dict.keys():
        filename = data_file.split('/')[-1]
        if json_filename in filename:
            out_dict[json_file].append(data_file)

with open(out_path, 'w') as f:
    json.dump(out_dict, f, indent=4)

In [ ]:
from utils.save_logs import save_jsons_to_data

save_jsons_to_data(
    file_infos_path='file_infos/test-01.json',
    jsons_to_data_path='file_infos/jsons_to_data/test-01.json'
)

**TO DO:**
> Manually correct the json in `out_path` to match each json to its correct data file (usually nifti).

## Change the file infos to match jsons

In [ ]:
# DEPRECATED
from utils.filename_reader import remove_extension

file_infos_path = 'file_infos/test.json'
corrected_file_infos_path = 'file_infos/test_corrected.json'
jsons_to_data_path = 'file_infos/jsons_to_data/test.json'

with open(file_infos_path, 'r') as f:
    file_infos = json.load(f)

with open(jsons_to_data_path, 'r') as f:
    jsons_to_data = json.load(f)

for json_file in jsons_to_data.keys():
    assert len(jsons_to_data[json_file])<=1
    if len(jsons_to_data[json_file])==1:
        matching_file = jsons_to_data[json_file][0]
        json_file_infos = file_infos[matching_file].copy()
        json_file_infos['extension'] = '.json'
        data_new_path = json_file_infos['new_path']
        json_new_path = remove_extension(data_new_path) + '.json'

        json_file_infos['new_path'] = json_new_path
        file_infos[json_file] = json_file_infos

with open(corrected_file_infos_path, 'w') as f:
    json.dump(file_infos, f, indent=4)

In [2]:
from utils.save_logs import correct_file_infos_with_matching_metadata

correct_file_infos_with_matching_metadata(
    file_infos_path = 'file_infos/test-01.json',
    jsons_to_data_path = 'file_infos/jsons_to_data/test-01.json',
    corrected_file_infos_path = 'file_infos/test-01_corrected.json'
)

# Save `paths.txt` and `recap.json`

## `paths.txt`

In [12]:
def write_paths_file(file_infos_path, out_path, old_prefix='', new_prefix=''):

    with open(file_infos_path,'r') as f:
        data = json.load(f)
    
    out_dirs = '/'.join(out_path.split('/')[:-1])
    try:
        os.makedirs(out_dirs)
        with open(out_path,'w') as f:
            for file in data.keys():
                f.write('~' + old_prefix + data[file]['old_path'] + ', ~/' + new_prefix + data[file]['new_path'])
                f.write('\n')
    except:
        with open(out_path,'a') as f:
            for file in data.keys():
                f.write('~' + data[file]['old_path'] + ', ~/' + data[file]['new_path'])
                f.write('\n')



In [1]:
from utils.save_logs import write_paths_file

file_infos_path = 'file_infos/test-01_corrected.json'
out_path = 'paths/test_01-corrected.txt'

write_paths_file(file_infos_path, out_path, old_prefix='', new_prefix='')

## `recap.json`

In [13]:
def write_general_recap_file(file_infos_path, out_path, new_prefix=''):

    with open(file_infos_path,'r') as f:
        file_infos = json.load(f)
    
    out_dirs = '/'.join(out_path.split('/')[:-1])
    out_data = {}

    try:
        with open(out_path,'w') as f:
            for file in file_infos.keys():
                type = file_infos[file]['type']
                new_path = new_prefix + file_infos[file]['new_path']
                sub = file_infos[file]['sub']
                if sub not in out_data.keys():
                    out_data[sub] = {}
                if type in out_data[sub].keys():
                    out_data[sub][type].append(new_path)
                else:
                    out_data[sub][type] = [new_path]
            json.dump(out_data, f, indent=4)
    except:
        try:
            os.makedirs(out_dirs)
        finally:
            with open(out_path,'w') as f:
                for file in file_infos.keys():
                    type = file_infos[file]['type']
                    new_path = new_prefix + file_infos[file]['new_path']
                    sub = file_infos[file]['sub']
                    if sub not in out_data.keys():
                        out_data[sub] = {}
                    if type in out_data[sub].keys():
                        out_data[sub][type].append(new_path)
                    else:
                        out_data[sub][type] = [new_path]
                json.dump(out_data, f, indent=4)
        


In [1]:
from utils.save_logs import write_general_recap_file

write_general_recap_file(
    file_infos_path='file_infos/test-01_corrected.json',
    out_path='recaps/recap_01.json',
    new_prefix=''
    )

In [15]:
def merge_general_recaps(input_files, out_path):
    out_data = {}
    for file in input_files:
        with open(file, 'r') as f:
            file_data = json.load(f)
        for sub in file_data.keys():
            if sub not in out_data.keys():
                out_data[sub] = file_data[sub]
            else:
                for file_type in file_data[sub].keys():
                    if file_type not in out_data[sub].keys():
                        out_data[sub][file_type] = file_data[sub][file_type]
                    else:
                        assert type(out_data[sub][file_type]) == list
                        for new_path in file_data[sub][file_type]:
                            if new_path not in out_data[sub][file_type]:
                                out_data[sub][file_type].append(new_path)
    try:
        with open(out_path, 'w') as f:
            json.dump(out_data, f, indent=4)
    except:
        out_dirs = '/'.join(out_path.split('/')[:-1])
        os.makedirs(out_dirs)
        with open(out_path, 'w') as f:
            json.dump(out_data, f, indent=4)
        

In [1]:
from utils.save_logs import merge_general_recaps

merge_general_recaps(
    input_files=[
        'recaps/recap_01.json',
        'recaps/recap_02.json'
    ],
    out_path='recaps/merged_recap_test.json'
)

# Copy and rename old files to their new paths

In [3]:
dbx.files_copy(
    from_path= '/source/REEVOID_PILOT_01_MRI_NIFTI/Resting_State/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.nii.gz',
    to_path='/target/sub_pilot/func/sub_pilot_resting_state_id-7_bold.nii.gz'
)

FileMetadata(client_modified=datetime.datetime(2025, 6, 17, 18, 6, 34), content_hash='e5556a7e2d867e5cd33e1c73bfea5b2561c51ba635f5766a2a895777e2e19d0a', export_info=NOT_SET, file_lock_info=NOT_SET, has_explicit_shared_members=NOT_SET, id='id:KLawZ41SnG8AAAAAAAAATQ', is_downloadable=True, media_info=NOT_SET, name='sub_pilot_resting_state_id-7_bold.nii.gz', parent_shared_folder_id=NOT_SET, path_display='/target/sub_pilot/func/sub_pilot_resting_state_id-7_bold.nii.gz', path_lower='/target/sub_pilot/func/sub_pilot_resting_state_id-7_bold.nii.gz', preview_url=NOT_SET, property_groups=NOT_SET, rev='016410af3178da8000000030e305703', server_modified=datetime.datetime(2025, 10, 13, 14, 19, 4), sharing_info=NOT_SET, size=85415620, symlink_info=NOT_SET)

In [1]:
from utils.dropbox_filesystem import get_all_paths
from tokens import ACCESS_TOKEN

all_paths = get_all_paths('', TOKEN= ACCESS_TOKEN, recursive=False, remove_source=False)

print(all_paths)

Creating a Dropbox object...
/target
/source
/requirements.txt
[]


In [1]:
from utils.dropbox_filesystem import get_all_paths
from tokens import ACCESS_TOKEN

all_paths = get_all_paths('/source', TOKEN= ACCESS_TOKEN, recursive=True, remove_source=True)

print(all_paths)

all_paths = get_all_paths('/source/', TOKEN= ACCESS_TOKEN, recursive=True, remove_source=True)

print(all_paths)

Creating a Dropbox object...
Creating a Dropbox object...
Creating a Dropbox object...
['/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.nii.gz', '/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.json', '/Reevoid_Martin_fMRI_interoperability.zip', '/PhysioLogFiles_fMRI_Martin.zip']
Creating a Dropbox object...
Creating a Dropbox object...
Creating a Dropbox object...
['/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.nii.gz', '/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.json', '/Reevoid_Martin_fMRI_interoperability.zip', '/PhysioLogFiles_fMRI_Martin.zip']


In [ ]:
file_infos_path = 'file_infos/test.json'

with open(file_infos_path, 'r') as f:
    file_infos = json.load(f)

for file in file_infos.keys():
    dbx.files_copy(
    from_path= '/source' + file_infos[file]["old_path"],
    to_path= '/target/' + file_infos[file]["new_path"]
)

In [2]:
str = '/source/REEVOID_PILOT_01_MRI_NIFTI/Resting_State/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.nii.gz'
print(str[len("/source"):])

/REEVOID_PILOT_01_MRI_NIFTI/Resting_State/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.nii.gz
